# Exercise 2: Temporal Hello World — Solution

Complete implementation of the Temporal Hello World workflow with an activity.

## Architecture Pattern 🏗️

```
Workflow Execution Request 👤
    ↓
Temporal Workflow (orchestration) 🎭
    ↓
Temporal Activity (unit of work) ⚙️
    ↓
[Process data, call APIs, etc.]
    ↓
Activity completes ✅
    ↓
Workflow returns result 📦
    ↓
Return to caller ✅
```

**Key Benefits:**
- ✅ Activities automatically retry on failure
- ✅ Workflow state persists across crashes
- ✅ Full execution history in Temporal UI

## Setup

Before running this solution, ensure you have:

1. **Temporal Server Running:**
   - Open `temporal_installation.ipynb` in the project root
   - Run all cells to install Temporal CLI and start the dev server
   - **Verify:** In Codespaces, go to the **Ports** tab → Find port **8233** → Click the **Globe icon** 🌐

2. **Dependencies Installed:**
   - Required packages should already be installed from the main setup


In [ ]:
import asyncio
import nest_asyncio
from datetime import timedelta, datetime
import pytz
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.worker import Worker, UnsandboxedWorkflowRunner

## 1. Define an Activity

In [3]:
@activity.defn
async def process_data(data: str) -> str:
    """Activity that simulates work by upper-casing the payload."""
    activity.logger.info("🔧 Activity started")
    await asyncio.sleep(1)
    result = f"Processed: {data.upper()}"
    activity.logger.info("✅ Activity completed")
    return result


## 2. Define a workflow

In [4]:
@workflow.defn
class HelloWorkflowTemporal:
    """Workflow that orchestrates the activity call."""

    @workflow.run
    async def run(self, name: str) -> str:
        workflow.logger.info("🚀 Workflow started")
        result = await workflow.execute_activity(
            process_data,
            args=[f"Hello {name}"],
            start_to_close_timeout=timedelta(seconds=10),
        )
        workflow.logger.info("✅ Workflow finished")
        return f"Workflow result: {result}"

## Run the Solution

Use the cell below to run the fully implemented solution script.

In [ ]:
# Execute the solution workflow
async def run_solution() -> None:
    print("\n🚀 Exercise 2: Temporal Hello World — Solution\n")
    client = await Client.connect("localhost:7233")
    task_queue = "hello-world-queue"
    
    # Get current time in EST and format workflow ID
    est = pytz.timezone('US/Eastern')
    now = datetime.now(est)
    workflow_id = f"02-workflow-{now.strftime('%a-%b-%d-%I%M%S').lower()}est"
    
    print(f"Workflow ID: {workflow_id}\n")

    async with Worker(
        client,
        task_queue=task_queue,
        workflows=[HelloWorkflowTemporal],  #sets the workflow type name in UI
        activities=[process_data],
        workflow_runner=UnsandboxedWorkflowRunner(),
    ):
        result = await client.execute_workflow(
            HelloWorkflowTemporal.run,
            "Temporal",
            id=workflow_id,
            task_queue=task_queue,
        )

    print(f"\n✅ Workflow Result: {result}\n")
    print(
        "View execution history: "
        f"http://localhost:8233/namespaces/default/workflows/{workflow_id}\n"
    )

# Run the workflow in a notebook-friendly way
try:
    asyncio.get_running_loop()
except RuntimeError:
    asyncio.run(run_solution())
else:
    nest_asyncio.apply()
    await run_solution()


🚀 Exercise 2: Temporal Hello World — Solution

Workflow ID: 02-workflow-fri-nov-07-011738est


✅ Workflow Result: Workflow result: Processed: HELLO TEMPORAL

View execution history: http://localhost:8233/namespaces/default/workflows/02-workflow-fri-nov-07-011738est



## Next Steps

**Ready for more?** Proceed to:

- **[Solution 3](../../solutions/03_durable_agent/solution.ipynb)** - Learn Temporal workflows and activities
- **[Solution 4](../../solutions/04_agent_routing/README.md)** - Combine agents + Temporal for production durability!